# 1 {-}

**The ASIA Bayesian network is a famous toy Bayesian network introduced in Lauritzen and Spiegelhalter, Journal of the Royal Statistical Society, Series B, 1988. The article is attached. See Figure 2 for the network and the start of section 4 for a description of the node meanings. The following website has a nice interface through which you can play with a fitted version of the model**

https://www.bayesserver.com/examples/networks/asia

**The network has 8 nodes. A sample $X \in R^8$ from the network is decomposed as follows**

$$X = (A,S,T,C,B,E,R,D) = \text{(Asia, smoker, tuberculosis, cancer, bronchitis, either, x-ray, dyspnoea)}$$


**Each $X_i \in \{0,1 \}$, e.g. a person is either a smoker (0) or not (1).**


# a {-}

**Table 1 in the paper provides the parameters to the model. Show that given these parameters, $P(X)$ is completely determined for all $X$ in the sample space of $X$.**

As noted above,

$$P(X)=P(A,S,T,C,B,E,R,D)$$

by the product rule, and considering the bayesian network and DAG property, we have

$$=P(A)P(T|A)P(E|T,C)P(R|E)P(D|E,B)P(C|S)P(B|S)P(S)$$.

# b {-}

**The paper presents the problem of computing $P(R = 1|A, S, D)$, with the intuition that knowledge of the probability of a positive X-ray will help a doctor determine if taking an X-ray is worthwhile. Compute $P(R = 1|A = 1,S = 0,D =1)$ in two ways**

**i. Construct a Metropolis-Hastings sampler (or if you want to try, a Gibbs sampler) for $X$. Use the sampler to estimate $P(R=1|A=1,S=0,D=1)$. (Use only 1 run of your MH sampler and don’t forget to allow a burn-in time).**


Let's first construct a sampler for $X$. Let $X$ be our random variable and let $\Omega$ be our sample space. In other words $\Omega= \{X \in R^{8}: X_i \in \{0,1 \}\}$. We want to be able to generate samples, $\omega$'s, from $\Omega$. The distribution of $X$ (our target distribution) is given in the paper.

Let's choose a symmetric proposal distribution $q$: We will propose $\omega'$ by randomly choosing one entry of the $8$ entries of $w$. So $q(\omega'|\omega)=q(\omega|\omega')=\frac{1}{8}.$


And so to generate a sample $\omega_{t+1}$ from our target distribution $P(X)$ we do the following:

1. Generate $\omega'$ from $q(\omega'|\omega_t)$
2. Generate $\hat{u}$ from  Uniform$(0,1)$
3. If $\hat{u} < \frac{p(\omega')q(\omega_t|\omega')}{p(\omega_t)q(\omega'|\omega_t)}=\frac{p(\omega')}{p(\omega_t)}$ accept $\omega'$ and set $\omega_{t+1}=\omega'$; Otherwise set $\omega_{t+1}=\omega_t$

Note that $\frac{p(\omega')}{p(\omega_t)}$ reduces to the ratio of the conditional distribution of the feature selected. For example, if $R$ was chosen at time $t$ and flipped from $0$ to $1$, then 

$$\frac{p(\omega')}{p(\omega_t)}=\frac{P(R=1|E)}{P(R=0|E)}$$



Let's first create the conditional probability table from Table 1 in the paper.

In [1]:
import numpy as np

# distribution for A
A = np.array([.01])

# cond distribution for S
S = np.array([0.05])

# cond table for T
T = np.array([0.05, 0.01])

# cond table for C
C = np.array([0.1, 0.01])

# cond table for B
B = np.array([0.6, 0.3])

# cond table for E, row is T and column is C
E = np.array([[1, 1],
        [1, 0]])

# cond table for R, row is E
R = np.array([0.98, 0.05])

# cond table for D, row is B and column is E
D = np.array([[0.9, 0.8],
            [0.7, 0.1]])

cond_tables = [A, S, T, C, B, E, R, D]

Let's also build a function that gets the parent positions in the 8 dimensional vector $X=(A,S,T,C,B,E,R,D)$.

In [2]:
def get_parents(i, X):
    """"""
    if i == 0:
        return
    elif i == 1:
        return
    elif i == 2:
        return X[0]
    elif i == 3:
        return X[1]
    elif i == 4:
        return X[1]
    elif i == 5:
        return (X[2], X[3])
    elif i == 6:
        return X[5]
    elif i == 7:
        return (X[4], X[5])

In [3]:
from tqdm import tqdm
def MetropolisHastings(N, w, indices=[0,1,2,3,4,5,6,7]):
    """Sampler via Metropolis Hastings
    Args:
        N: number of iterations
        w: matrix of shape (8, )
        indices: coordinates to randomly choose from in proposal distribution
    Returns:
        samples: list of shape (N, 8)
    """
    samples = [w]
    for _ in tqdm(range(N)):
        index = np.random.choice(indices)
        if (index == 0) or (index == 1):
            p = np.abs(w[index] - cond_tables[index].item())
        else:
            p = np.abs(w[index] - cond_tables[index][get_parents(index, w)])
        mhr = (1 - p) / (p + 10**-20) # adding 10**-20 for numerical stability
        if np.random.uniform(0, 1) < mhr:
            w[index] = 1 - w[index]
        samples.append(w.copy())
    return samples

We want $P(R=1|A=1,S=0,D=1)$.

To find the above probability, let's sample from 

$$P(R,T,C,B,E|A=1,S=0,D=1) = \frac{P(R,T,C,B,E,A=1,S=0,D=1)}{\alpha}$$

where $\alpha$ is the normalizing constant.

We sample from $P(R,T,C,B,E|A=1,S=0,D=1)$ by sampling from $P(X)$ as discussed above and keeping fixed $A, S, D$ at $A=1,S=0,D=1$ -- we choose randomly between the remaining features so our proposal distribution, still symmetric, changes to $q(\omega'|\omega)=q(\omega|\omega')=\frac{1}{5}.$ 

In [4]:
w_start = np.array([1,0,1,1,1,1,1,1])
N = 10**6
samples = MetropolisHastings(N, w_start, indices=[2,3,4,5,6])

100%|██████████| 1000000/1000000 [00:18<00:00, 54281.26it/s]


Now to estimate $P(R=1|A=1,S=0,D=1)$ we can invoke the Law of Large Numbers. 

Let $H$ be an indicator random variable that equals $1$ if $R=1$ and $0$ otherwise.

$$H=\begin{cases} 
      1 & \text{if}~ R=1|A=1,S=0,D=1 \\
      0 & \text{if}~ R=0|A=1,S=0,D=1
   \end{cases}
$$

So,

$$E[H]=1\cdot P\{R=1|A=1,S=0,D=1\}+0 \cdot P\{R=0|A=1,S=0,D=1\}=P\{R=1|A=1,S=0,D=1\}$$

And so now we can approximate $E[H]$ with $\frac{1}{N}\sum^{N}_{n=1}\mathbb{1}_{\omega_{nR}=1}$ for an appropriate $N$ runs of the Markov Chain, excluding the burn in time. Note $\omega_{nR}$ is the entry corresponding to the $R$ feature of the $n$th sample. 

In [5]:
# finding the sample mean
# including burn in at 0.3*N
l = np.mean(samples[int(0.3*len(samples)):], axis=0)

In [6]:
l[6]

0.9257134286571344

And so, using Markov Chain Monte Carlo, $P(R=1|A=1,S=0,D=1) \approx 0.9257134286571344$

**ii. Using the relation,**

$$P(R = 1|A = 1,S = 0,D = 1)=\frac{P(R = 1,A = 1,S = 0,D = 1)}{P (A = 1, S = 0, D = 1)}$$

**write down formulas for both the numerator and the denominator (these will be summations) and use R/Python to evaluate the formulas and, in turn, compute $P(R = 1|A = 1,S = 0,D = 1)$. (Note: The small size of the network allows us to apply this explicit approach, but it would not be possible for a larger network)**

$$P(R = 1|A = 1,S = 0,D = 1)$$

$$=\frac{P(R = 1,A = 1,S = 0,D = 1)}{P (A = 1, S = 0, D = 1)}$$

$$=\frac{\sum_{t=0}^{1}\sum_{c=0}^{1}\sum_{b=0}^{1}\sum_{e=0}^{1}P(R=1,T=t,C=c,B=b,E=e,A=1,S=0,D=1)}{\sum_{t=0}^{1}\sum_{c=0}^{1}\sum_{b=0}^{1}\sum_{e=0}^{1}\sum_{r=0}^{1}P(R=r,T=t,C=c,B=b,E=e,A=1,S=0,D=1)}$$




In [9]:
def helper_calc(R, A=1, S=0, D=1):
    """"""
    p = 0
    for T in range(0,2):
        for C in range(0,2):
            for B in range(0,2):
                for E in range(0,2):
                    p = p + cond_tables[A].item()*cond_tables[S].item()*\
                    np.abs(T-cond_tables[2][A])*np.abs(C-cond_tables[3][S])*\
                    np.abs(B-cond_tables[4][S])*np.abs(E-cond_tables[5][T, C])*\
                    np.abs(R-cond_tables[6][E])*np.abs(D-cond_tables[7][B, E])
    
    return p

In [10]:
helper_calc(1) / (helper_calc(1) + helper_calc(0))

0.9092072434607645

And so, using Markov Chain Monte Carlo, $P(R=1|A=1,S=0,D=1) = 0.9092072434607645$, which is close to what we estimated via Markov Chain Monte Carlo.

# 2 {-}
**A third method we could implement to compute smoothing probabilities such as $P(R = 1|A = 1,S = 0,D = 1)$ is a forward-backward iteration. However, we have only considered this approach for a tree, and the $\delta$ (dsypnoea) and $\epsilon$ (either) nodes have two parents, Instead, consider the tree shown below. For the sake of time, we’ll only consider writing down one step in these iterations, without actually implementing the iteration. Assume that each node has as its states, $\{1, 2, . . . , m\}$.**

![](tree.jpg)

# a {-}

**Consider the backward iteration and suppose that we know $\beta_t(i)$ when the node $t$ is either node $9$ or node $10$. Derive a formula for $\beta_t(i)$ for the case of $t$ equal to node $6$.**


$$\beta_6(i)$$

$$=P(V_6^+|X_6=i)$$

$$=\sum_{j=1}^{m}\sum_{j'=1}^{m}P(V_9^+, V_{10}^+, X_9=j, X_{10}=j'|X_6=i)$$

$$=\sum_{j=1}^{m}\sum_{j'=1}^{m}P(V_9^+, V_{10}^+|X_6=i, X_9=j, X_{10}=j')P(X_9=j, X_{10}=j'|X_6=i)$$

$$=\sum_{j=1}^{m}\sum_{j'=1}^{m}P(V_9^+|V_{10}^+,X_6=i, X_9=j, X_{10}=j')P(V_{10}^+|X_6=i, X_9=j, X_{10}=j')$$

$$P(X_9=j|X_{10}=j',X_6=i)P(X_{10}=j'|X_6=i)$$

$$=\sum_{j=1}^{m}\sum_{j'=1}^{m}P(V_9^+|X_9=j)P(V_{10}^+|X_{10}=j')P(X_9=j|X_6=i)P(X_{10}=j'|X_6=i)~\text{by the DAG properties}$$

$$=\sum_{j=1}^{m}\sum_{j'=1}^{m}\beta_9(j)\beta_{10}(j')P(X_9=j|X_6=i)P(X_{10}=j'|X_6=i)$$

# b {-}

**Suppose we have completed the backward iteration and consider the forward iteration. Suppose we know $\alpha_t(i)$ for $t$ as node $6$. Derive a formula for $\alpha_t(i)$ for $t$ equal to node $9$.**

$$\alpha_9(i)$$

$$=P(X_9=i, V_9^-)$$

$$=P(X_9=i, V_{10}^+, V_6^-)~\text{since}~V_9^-=V_{10}^+\cup V_6^-$$

$$=\sum_{j=1}^{m}P(X_9=i, V_{10}^+, V_6^-, X_6=j)$$

$$=\sum_{j=1}^{m}P(X_9=i, V_{10}^+| V_6^-, X_6=j)P(V_6^-, X_6=j)$$

$$=\sum_{j=1}^{m}\alpha_6(j)P(X_9=i, V_{10}^+| X_6=j)~\text{by DAG properties}$$

$$=\sum_{j=1}^{m}\sum_{k=1}^{m}\alpha_6(j)P(X_9=i, V_{10}^+, X_{10}=k|X_6=j)$$

$$=\sum_{j=1}^{m}\sum_{k=1}^{m}\alpha_6(j)P(X_9=i| V_{10}^+, X_{10}=k,X_6=j)$$

$$P(V_{10}^+| X_{10}=k,X_6=j)P(X_{10}=k|X_6=j)$$

$$=\sum_{j=1}^{m}\sum_{k=1}^{m}\alpha_6(j)P(X_9=i|X_6=j)P(V_{10}^+| X_{10}=k)P(X_{10}=k|X_6=j)~\text{by DAG properties}$$

$$=\sum_{j=1}^{m}\sum_{k=1}^{m}\alpha_6(j)P(X_9=i|X_6=j)\beta_{10}(k)P(X_{10}=k|X_6=j)$$